### 01. Testando conexão e integração como o Storage (MinIO)

#### 01.1 - Criando Client para conexão com o Storage MinIO

In [12]:
# IMPORTS AND LIBRARIES

import os
import json
import delta
import requests
from datetime import datetime

import boto3
import s3fs

import logging

# Configuração do logger
logger = logging.getLogger("minio_logger")
logger.setLevel(logging.INFO)

# Configurando o formato do log
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)


In [14]:
# Variáveis Globais e de ambiente para o projeto.


bucket_name = "bank-databr"

os.environ["MINIO_KEY"] = "developer"
os.environ["MINIO_SECRET"] = "developer01"
os.environ["MINIO_ENDPOINT"] = "http://minio:9000"



In [15]:

s3_client = boto3.client(
    's3',
    endpoint_url = os.environ.get("MINIO_ENDPOINT"),
    aws_access_key_id = os.environ.get("MINIO_KEY"),
    aws_secret_access_key = os.environ.get("MINIO_SECRET")
)


#### 01.2 - Gravando Arquivo de Exemplo

In [17]:

class DataModel:
    """ Simplifica e Padroniza objetos a serem gravados no Storage MinIO S3 """

    def __init__(self, bucket_name: str, destination_file_path: str, file_content: str):
        self.bucket_name = bucket_name
        self.destination_file_path = destination_file_path
        self.file_content = file_content        


# Arquivo de Exemplo para testes no Client MinIO
teste_data = DataModel(    
    bucket_name = bucket_name,
    destination_file_path = f"/landing/bacen/teste.json",
    file_content = json.dumps({"message": "hello world"})    
)


def write_file(s3_client: boto3.client, data_params: DataModel) -> None:
    """ Grava arquivo no Storage 
        Args:
            s3_client: Client de conexão/integração com o storage
            data_params: Objeto a ser gravado no storage padrão DataModel
    """

    logger.info("Gravando Arquivo")
    
    try:        

        s3_client.put_object(
            Bucket = data_params.bucket_name, 
            Key = data_params.destination_file_path,
            Body = data_params.file_content
        )
    
        logger.info(f"\t* Arquivo {data_params.destination_file_path} gravado com sucesso")
        
    except Exception as e:
        logger.error(f"\t* Erro ao tentar gravar arquivo {data_params.bucket_name}{data_params.destination_file_path}, error: {e}")


        
# Gravando um novo Arquivo no Storage
write_file(s3_client, teste_data)


2025-01-19 17:12:50,071 - INFO - Gravando Arquivo
2025-01-19 17:12:50,071 - INFO - Gravando Arquivo
2025-01-19 17:12:50,131 - INFO - 	* Arquivo /landing/bacen/teste.json gravado com sucesso
2025-01-19 17:12:50,131 - INFO - 	* Arquivo /landing/bacen/teste.json gravado com sucesso


#### 01.3 - Listando todos os arquivos no Bucket informado

In [19]:

def list_files_in_bucket(s3_client:boto3.client, destination_file_path:str) -> list:
    """ Lista todos os arquivos no caminho indicado dentro do Storage 
        Args:
            s3_client: Client de integração/conexão com o storage 
            path: caminho a ser listado dentro do bucket
    """

    logger.info(f"Listando todos os arquivos em {destination_file_path}")

    bucket_name, *prefix_parts = destination_file_path.replace("s3://", "").split("/", 1)
    prefix = prefix_parts[0] if prefix_parts else ""
   
    try:
        paginator = s3_client.get_paginator("list_objects_v2")
        for page in paginator.paginate(Bucket=bucket_name, Prefix = prefix):
    
            if "Contents" in page:
                for obj in page['Contents']:
                    yield obj['Key'] # retorna o nome do arquivo
            else:
                log.warn("\t* Nenhum arquivo encontrado")
                
    except Exception as e:
        logger.error(f"Erro ao lista arquivos: {e}")

landing_path_dir = f"{bucket_name}/landing/bacen"

# Listando todos os arquivos na pasta raiz do Storage
print('\nListando todos os arquivos na Landing')
for file in list_files_in_bucket(s3_client, landing_path_dir):
    print(f"\t* {file}")


2025-01-19 17:13:52,176 - INFO - Listando todos os arquivos em bank-databr/landing/bacen
2025-01-19 17:13:52,176 - INFO - Listando todos os arquivos em bank-databr/landing/bacen



Listando todos os arquivos na Landing
	* landing/bacen/cartoes_trimestral/data_18_01_2025_17_39_57.json
	* landing/bacen/meios_pagamentos_mensal/data_18_01_2025_17_39_57.json
	* landing/bacen/meios_pagamentos_trimestral/data_18_01_2025_17_39_57.json
	* landing/bacen/teste.json


#### 01.4 - Deletando arquivo de Testes

In [20]:

def remove_files_in_bucket(s3_client:boto3.client, file_path:str, file_name:str) -> None:
    """ Remove arquivo, se existir, no folder indicado
        Args:
            s3_client: Client de integração/conexão com o storage
            file_path: Caminho do arquivo a ser deletado no bucket/folder indicado
            file_name: Nome do arquivo a ser deletado no bucket/folder indicado
    
    """
    logger.info(f"Removendo arquivo: {file_path}/{file_name}")

    try:
        s3_client.delete_object(Bucket=file_path, Key=file_name)
        logger.info(f"\t* Aquivo {file_path}/{file_name} deletado com sucesso!")

    except Exception as e:
        logger.error(f"\t* Erro ao tentar deletar arquivo: {file_path}/{file_name}, error: {e}")

# Deletando arquivo no Storage
remove_files_in_bucket(s3_client, bucket_name, "teste.json")


2025-01-19 17:14:01,443 - INFO - Removendo arquivo: bank-databr/teste.json
2025-01-19 17:14:01,443 - INFO - Removendo arquivo: bank-databr/teste.json
2025-01-19 17:14:01,473 - INFO - 	* Aquivo bank-databr/teste.json deletado com sucesso!
2025-01-19 17:14:01,473 - INFO - 	* Aquivo bank-databr/teste.json deletado com sucesso!
